In [ ]:
pip install rdkit-pypi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 36.8 MB 29 kB/s 


In [ ]:
import numpy as np
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, mean_squared_error, r2_score
import matplotlib.pyplot as plt

def read_data(filename):
    f = open('sr-p53.smiles', 'r')
    contents = f.readlines()

    smiles = []
    labels = []
    for i in contents:
        smi = i.split()[0]
        label = int(i.split()[2].strip())

        smiles.append(smi)
        labels.append(label)

    num_total = len(smiles)
    rand_int = np.random.randint(num_total, size=(num_total,))
    
    return np.asarray(smiles)[rand_int], np.asarray(labels)[rand_int], num_total

In [ ]:
smiles, label, num_total = read_data('sr-p53.smiles') #실제로 읽어오는 단계

FileNotFoundError: ignored

In [ ]:
num_total

NameError: ignored

In [ ]:
fps_total = []
tox_total = []

for i in range(num_total): #600개에 대한 fingerprint만들기, 일단 범위부터.
    smi = smiles[i].split()[0]
    tt = label[i]
    m = Chem.MolFromSmiles(smi)
    fp = Chem.AllChem.GetMorganFingerprintAsBitVect(m,2, nBits=2048, useChirality=False) #fingerprint
    arr = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(fp,arr)
    fps_total.append(arr)
    tox_total.append(tt)

In [ ]:
fps_total = np.asarray(fps_total)
tox_total = np.asarray(tox_total)

In [ ]:
num_total = fps_total.shape[0]
num_train = int(num_total*0.8) #int는 정수로 바꾸는 거

num_total, num_train, (num_total-num_train)

(600, 480, 120)

In [ ]:
fps_train = fps_total[0:num_train]
tox_train = tox_total[0:num_train]
fps_test = fps_total[num_train:]
tox_test = tox_total[num_train:]

In [ ]:
#kernel in ('linear', 'rbf', 'poly'):
clf = SVC(kernel='linear', gamma=10)
clf.fit(fps_train, tox_train)

SVC(gamma=10, kernel='linear')

In [ ]:
tox_pred = clf.predict(fps_test)
r2 = r2_score(tox_test, tox_pred)
mse = mean_squared_error(tox_test, tox_pred)
r2, mse 

(0.45930723739791623, 0.13333333333333333)